In [1]:
import numpy as np
import pandas as pd
import copy

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pdb

In [118]:
class Trajectory:
    
    def __init__(self, array):
        self.array = array # should be a 2d array
        *self.x, self.w, self.y = self.array.T
    
    def append(self, item):
        self.array = np.concatenate([self.array, item.reshape(1,-1)])
        *self.x, self.w, self.y = self.array.T
        
    def copy(self):
        return Trajectory(copy.deepcopy(self.array))
    
    def __getitem__(self, t):
        return Trajectory(self.array[t,:].reshape(1,-1))
    
    def gen_covar(self):
        return (xt for xt in zip(*self.x))
    
    def gen_outcome(self):
        return (yt for yt in self.y)
    
    @property
    def df(self):
        col_names = ['x0', 'x1', 'x2', 'w', 'y']
        return pd.DataFrame({
            name:col for name, col in 
            zip(col_names, self.array.T)
        })


class Patient:
# could probably get rid of the state attr and move it into traj    
    
    def __init__(self,  
                 init_state = np.zeros(5), 
                 T = 0,
                 gen_covar = lambda patient: (0.0, 0, 0),
                 gen_alert = lambda patient: 0.0, 
                 gen_outcome = lambda patient: 0.0):
    
        self.state = init_state
        self.T = T
        
        self.trajectory = Trajectory(self.state.reshape(1,-1))
        self.alerts = []
        self.outcomes = []
        
        self.gen_covar = gen_covar
        self.gen_outcome = gen_outcome
        self.gen_alert = gen_alert
        
        for self.t in range(T-1):
            self.state = np.array([*self.gen_covar(self), 
                                   self.gen_alert(self), 
                                   self.gen_outcome(self)])
            self.trajectory.append(self.state)
            *x, w, y = self.state
            if w:
                self.alerts.append(self.t + 1)
            if y:
                self.outcomes.append(self.t + 1)
                
    def lookahead_traj(self, outcome_lookahead):
        traj_copy = self.trajectory.copy()
        if self.outcomes:
            outcome_in_lookahead_times = np.unique(np.concatenate([t-np.array(range(outcome_lookahead)) for t in self.outcomes]))
            outcome_in_lookahead_times = outcome_in_lookahead_times[outcome_in_lookahead_times>=0]
            traj_copy.y[outcome_in_lookahead_times] = 1.0
        return traj_copy
    
    @property
    def df(self, outcome_lookahead=None):
        return self.lookahead_traj(outcome_lookahead).df if outcome_lookahead else self.trajectory.df
    
    def aggregated_time_df(self, outcome_lookahead, **kwargs):
        return self.lookahead_traj(outcome_lookahead).df
    
    def fixed_time_df(self, outcome_lookahead, t):
        return self.lookahead_traj(outcome_lookahead)[t].df
    
    def first_alert_df(self):
        t = self.alerts[0] if self.alerts else 0
        return self.lookahead_traj(outcome_lookahead=self.T)[t].df
    
    def score(self, policy):
        covar_generator = self.trajectory.gen_covar()
        outcome_generator = self.trajectory.gen_outcome()
        _,_ = next(covar_generator), next(outcome_generator)
        return Patient(init_state = self.trajectory.array[0], T = self.T,
                 gen_covar = lambda patient: next(covar_generator),
                 gen_alert = policy,
                 gen_outcome = lambda patient: next(outcome_generator))

    
class Cohort():
    
    def __init__(self, n=None, patients=None, **kwargs):
            self.n = n
            self.kwargs = kwargs
            if patients:
                self.patients = patients
                self.n = len(patients)
            else:
                self.patients = [Patient(**kwargs) for i in range(n)]
        
    def score(self, policy):
        return Cohort(patients=[p.score(policy) for p in self.patients], **copy.deepcopy(self.kwargs))
    
    def clone(self, policy=lambda p: 0):
        kwargs = copy.deepcopy(self.kwargs)
        if 'gen_alert' in kwargs.keys():
            del kwargs['gen_alert']
        return Cohort(n=self.n, gen_alert=policy, **kwargs)
    
    def training_dataset(self, outcome_lookahead):
        return pd.concat([p.aggregated_time_df(outcome_lookahead) for p in self.patients])
            
    def eval_aggregated_time(self, outcome_lookahead):
        dataset = self.training_dataset(outcome_lookahead)
        return confusion_matrix(dataset.y, dataset.w, labels=[0,1])
    
    def eval_fixed_time(self, outcome_lookahead, t):
        dataset = pd.concat([p.fixed_time_df(outcome_lookahead, t) for p in self.patients])
        return confusion_matrix(dataset.y, dataset.w, labels=[0,1])

    def eval_first_alert(self):
        dataset = pd.concat([p.first_alert_df() for p in self.patients])
        return confusion_matrix(dataset.y, dataset.w, labels=[0,1])
    
    def __getitem__(self, index):
        new_cohort = Cohort(0)
        new_cohort.patients = [p for i,p in enumerate(self.patients) if i in index]
        return new_cohort
    
    def fit_risk_model(self, outcome_lookahead, alert_time=None, alert_after_alert=False, alert_after_outcome=False):
        training_data = self.training_dataset(outcome_lookahead)
        X = np.array(training_data[['x0','x1','x2']])
        Y = np.array(training_data[['y']]).ravel()
        risk_model = LogisticRegression(solver='lbfgs').fit(X, Y)

        def risk_model_system(patient):
            *x, w, y = patient.state
            if (alert_time and alert_time == patient.t) or alert_time is None:        
                if patient.alerts and not alert_after_alert:
                    return 0.0
                if patient.outcomes and not alert_after_outcome:
                    return 0.0
                else:
                    return risk_model.predict_proba(np.array(x).reshape(1, -1))[:,1]
            else:
                return 0.0

        return risk_model_system
    
    def counts(self):
        alert_count = sum(len(p.alerts) for p in self.patients)
        outcome_count = sum(len(p.outcomes) for p in self.patients)
        return np.array([alert_count, outcome_count])

# Sims

In [157]:
def gen_outcome(patient):
    *x, w, y = patient.state
    if patient.outcomes: # only one outcome per patient
        return 0.0
    else:
        return float(x[2] > 1)

def gen_covar(patient): # http://www.physics.udel.edu/~bnikolic/teaching/phys660/numerical_ode/node1.html
    *x, w, y = patient.state
    dt = 0.2
    xnew = [0,0,0]
    xnew[0] = np.random.normal(0.5,2) # "acceleration" 
    xnew[1] = x[1] + x[0]*dt # "velocity"
    xnew[2] = x[2] + x[1]*dt # "position"

    if w: # apply a force
        xnew[0] = x[0] - 20
#         xnew[2] = -np.inf

    return xnew

init = np.zeros((5,))

In [158]:
def run_simulation(
                    obs_outcome_lookahead = 10,
                    n = 1000,
                    T = 20,
                    risk_thresholds = [0, 1],
                    alert_time = None,
                    tstar = 10
                ):

    training_cohort = Cohort(n=training_n, 
                             init_state=init, T=T, 
                             gen_covar=gen_covar, 
                             gen_outcome = gen_outcome)
    risk_model = training_cohort.fit_risk_model(obs_outcome_lookahead, alert_time)
    test_cohort = training_cohort.clone()

    eval_results = []
    rct_results = []
    for c in risk_thresholds:
        policy = lambda p: float(risk_model(p)>c)

        scored_test_cohort = test_cohort.score(policy)

        agg = scored_test_cohort.eval_aggregated_time(obs_outcome_lookahead)
        eval_results.append({'Evaluation':'Aggregated Time', 'Threshold':c, 'True Positives':agg[1,1], 'Positives':np.sum(agg[:,1])})
        
        fixed = scored_test_cohort.eval_fixed_time(T, tstar)
        eval_results.append({'Evaluation':'Fixed Time', 'Threshold':c, 'True Positives':fixed[1,1], 'Positives':np.sum(fixed[:,1])})
        
        first = scored_test_cohort.eval_first_alert()
        eval_results.append({'Evaluation':'First Alert', 'Threshold':c, 'True Positives':first[1,1], 'Positives':np.sum(first[:,1])})

        prospective_cohort = test_cohort.clone(policy=policy)
        n_trt = prospective_cohort.counts()
        n_ctrl = test_cohort.counts()
        outcome_reduction = n_ctrl[1] - n_trt[1]
        alert_workload = n_trt[0]

        rct_results.append({'Threshold':c, 'Prevented Outcomes':outcome_reduction, 'Alerts':alert_workload})

    return pd.DataFrame(eval_results), pd.DataFrame(rct_results)

In [159]:
eval_results, rct_results = run_simulation(            
            obs_outcome_lookahead = 5,
            n = 500,
            T = 20,
            risk_thresholds = [0.2,0.4,0.6,0.8],
            alert_time = None,
            tstar = 10)

In [160]:
print(eval_results.to_latex(index=False))

\begin{tabular}{lrrr}
\toprule
      Evaluation &  Threshold &  True Positives &  Positives \\
\midrule
 Aggregated Time &        0.2 &             287 &        929 \\
      Fixed Time &        0.2 &              31 &         44 \\
     First Alert &        0.2 &             705 &        929 \\
 Aggregated Time &        0.4 &             383 &        522 \\
      Fixed Time &        0.4 &              33 &         35 \\
     First Alert &        0.4 &             423 &        522 \\
 Aggregated Time &        0.6 &             137 &        191 \\
      Fixed Time &        0.6 &              15 &         16 \\
     First Alert &        0.6 &             142 &        191 \\
 Aggregated Time &        0.8 &              26 &         34 \\
      Fixed Time &        0.8 &               3 &          3 \\
     First Alert &        0.8 &              27 &         34 \\
\bottomrule
\end{tabular}



In [161]:
print(rct_results.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
 Threshold &  Prevented Outcomes &  Alerts \\
\midrule
       0.2 &                 697 &     936 \\
       0.4 &                 337 &     539 \\
       0.6 &                  72 &     236 \\
       0.8 &                   7 &      29 \\
\bottomrule
\end{tabular}

